### Initialize

In [34]:

import requests
import pandas as pd
import os
import sys
from datetime import datetime

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_client as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path

base_url = get_base_url()
root_path = get_root_path()
example_name = sys.path[0].split(os.sep)[-2]


def execute_search_query(query):
    r = requests.post(
        f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[
                ['firstName', 'lastName', 'age']]
            df.rename(columns={'firstName': 'First Name',
                      'lastName': 'Last Name', 'age': 'Age'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


### Search for a Person where the Location of its is in London
--> It is searched for a person where a location of a person is related to London

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relLocation', 'location', 'city']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='London'))
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for a Person where the case is about money laundering
--> It is searched for a person who is involved in money laundering 

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relCases', 'cases', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='*money*laundering*'))  # String search with spaces and semi colons not possible
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for Persons where the case is about money laundering and the leads includes "payments"
--> This is an enhancment of the query before. It is searched for the payments of the money laundering

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relCases', 'cases', 'relLeads', 'lead', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Kennedy*'))  # String search with spaces and semi colons not possible
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for the time range of the payments when the person did the money laundering
--> This is an enhancement of the query before. It is searched for the time ranges when the payments were done

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relCases', 'cases', 'relLeads', 'lead', 'entityTime']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[2013-01-01 2019-12-12]'))  # String search with spaces and semi colons not possible
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for a case where the actvity was a phone intercept between J Kennedy and B Ramage

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope='Cases',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relActivity', 'activity', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Phone*intercept*'))  # String search with spaces and semi colons not possible
        ]))


r = requests.post(
    f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[so.dict()])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:
        df = pd.DataFrame.from_dict(res['value'])[
            ['entityDesc', 'entityTime', 'caseStartDate', 'caseSummary', 'priority']]
        df.rename(columns={
            'entityDesc': 'Descritpion', 'entityTime': 'Time', 'caseStartDate': 'Start Date', 'caseSummary': 'Summary', 'priority': 'Priority'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())


## Search for the objects P Lomez of a activity

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope='Activity',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relObject', 'object', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='P*lomez*'))  # String search with spaces and semi colons not possible
        ]))


r = requests.post(
    f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[so.dict()])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:
        df = pd.DataFrame.from_dict(res['value'])[
            ['entityDesc', 'entityTime']]
        df.rename(columns={
            'entityDesc': 'Descritpion', 'entityTime': 'Time'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())


## Search for a person who has an incident with "Threat of violence" at the location "Dingley Apartments, Braddon"

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relIncident', 'incident', 'entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Threat*violence*')),  # String search with spaces and semi colons not possible
            esh.Comparison(
                property=esh.Property(
                    property=['relIncident', 'incident', 'location']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Dingley*Apartments*')
            )
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for witness who reported the incident the assault JKennedy on J Roberts

In [39]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(
                    property=['relIncident', 'relTypeCode', 'description']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Witness')),  # String search with spaces and semi colons not possible
            esh.Comparison(
                property=esh.Property(
                    property=['relIncident','incident','entityDesc']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='*Kennedy*on*'))    

        ]))

execute_search_query(so.dict(exclude_none=True))

Total count: 0
No results
